<a href="https://colab.research.google.com/github/luanps/pyserini/blob/master/Pyserini_LTR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## LTR - Learning-To-Rank Reranking Models for MS MARCO Passage


### Install Dependencies

In [1]:
from google.colab import auth
auth.authenticate_user()

In [2]:
!gsutil cp gs://luanps/information_retrieval/pyserini/bm25/run.msmarco-passage.bm25tuned.txt .

Copying gs://luanps/information_retrieval/pyserini/bm25/run.msmarco-passage.bm25tuned.txt...
\ [1 files][126.9 MiB/126.9 MiB]                                                
Operation completed over 1 objects/126.9 MiB.                                    


In [3]:
!git clone https://github.com/luanps/pyserini.git  --recurse-submodules

Cloning into 'pyserini'...
remote: Enumerating objects: 5237, done.
remote: Counting objects: 100% (80/80), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 5237 (delta 32), reused 23 (delta 16), pack-reused 5157
Receiving objects: 100% (5237/5237), 2.40 MiB | 19.07 MiB/s, done.
Resolving deltas: 100% (3665/3665), done.
Submodule 'tools' (https://github.com/castorini/anserini-tools.git) registered for path 'tools'
Cloning into '/content/pyserini/tools'...
remote: Enumerating objects: 353, done.        
remote: Counting objects: 100% (110/110), done.        
remote: Compressing objects: 100% (84/84), done.        
remote: Total 353 (delta 54), reused 63 (delta 25), pack-reused 243        
Receiving objects: 100% (353/353), 23.60 MiB | 20.64 MiB/s, done.
Resolving deltas: 100% (138/138), done.
Submodule path 'tools': checked out '63ceeab1dd94c1221f29b931d868e8fab67cc25c'


In [ ]:
!pip install pyserini 

In [ ]:
!pip install -U pip setuptools wheel
!pip install -U spacy
!python -m spacy download en_core_web_sm

In [6]:
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 54.3 MB/s eta 0:00:00


In [7]:
!wget https://rgw.cs.uwaterloo.ca/JIMMYLIN-bucket0/pyserini-models/model-ltr-ibm.tar.gz -P collections/msmarco-ltr-passage/
!tar -xzvf collections/msmarco-ltr-passage/model-ltr-ibm.tar.gz -C collections/msmarco-ltr-passage/

!wget https://rgw.cs.uwaterloo.ca/JIMMYLIN-bucket0/pyserini-models/model-ltr-msmarco-passage-mrr-v1.tar.gz -P runs/
!tar -xzvf runs/model-ltr-msmarco-passage-mrr-v1.tar.gz -C runs

--2022-03-03 16:29:29--  https://rgw.cs.uwaterloo.ca/JIMMYLIN-bucket0/pyserini-models/model-ltr-ibm.tar.gz
Resolving rgw.cs.uwaterloo.ca (rgw.cs.uwaterloo.ca)... 129.97.167.73
Connecting to rgw.cs.uwaterloo.ca (rgw.cs.uwaterloo.ca)|129.97.167.73|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 252776626 (241M) [application/gzip]
Saving to: ‘collections/msmarco-ltr-passage/model-ltr-ibm.tar.gz’

model-ltr-ibm.tar.g 100%[===================>] 241.07M  70.0MB/s    in 3.7s    

2022-03-03 16:29:33 (65.3 MB/s) - ‘collections/msmarco-ltr-passage/model-ltr-ibm.tar.gz’ saved [252776626/252776626]

ibm_model/
ibm_model/url_unlemm/
ibm_model/url_unlemm/source.vcb
ibm_model/url_unlemm/target.vcb
ibm_model/url_unlemm/output.t1.5.bin
ibm_model/body/
ibm_model/body/source.vcb
ibm_model/body/target.vcb
ibm_model/body/output.t1.5.bin
ibm_model/text_bert_tok/
ibm_model/text_bert_tok/source.vcb
ibm_model/text_bert_tok/target.vcb
ibm_model/text_bert_tok/output.t1.5.bin
ibm_model/

In [8]:
#Download MSMARCO dataset

!mkdir collections/msmarco-passage
!wget https://msmarco.blob.core.windows.net/msmarcoranking/collectionandqueries.tar.gz -P collections/msmarco-passage
# Alternative mirror:
# wget https://www.dropbox.com/s/9f54jg2f71ray3b/collectionandqueries.tar.gz -P collections/msmarco-passage
!tar xvfz collections/msmarco-passage/collectionandqueries.tar.gz -C collections/msmarco-passage

--2022-03-03 16:29:50--  https://msmarco.blob.core.windows.net/msmarcoranking/collectionandqueries.tar.gz
Resolving msmarco.blob.core.windows.net (msmarco.blob.core.windows.net)... 20.150.34.4
Connecting to msmarco.blob.core.windows.net (msmarco.blob.core.windows.net)|20.150.34.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1057717952 (1009M) [application/gzip]
Saving to: ‘collections/msmarco-passage/collectionandqueries.tar.gz’

collectionandquerie 100%[===================>]   1009M  2.01MB/s    in 8m 36s  

2022-03-03 16:38:27 (1.95 MB/s) - ‘collections/msmarco-passage/collectionandqueries.tar.gz’ saved [1057717952/1057717952]

collection.tsv
qrels.dev.small.tsv
qrels.train.tsv
queries.dev.small.tsv
queries.dev.tsv
queries.eval.small.tsv
queries.eval.tsv
queries.train.tsv


In [9]:
!cp -r pyserini/tools/ .
!cp -r pyserini/scripts .

In [ ]:
'''!mkdir collections/msmarco-ltr-passage/

!python pyserini/scripts/ltr_msmarco/convert_queries.py \
  --input collections/msmarco-passage/queries.eval.small.tsv \
  --output collections/msmarco-ltr-passage/queries.eval.small.json 

!python pyserini/scripts/ltr_msmarco/convert_queries.py \
  --input collections/msmarco-passage/queries.dev.small.tsv \
  --output collections/msmarco-ltr-passage/queries.dev.small.json

!python pyserini/scripts/ltr_msmarco/convert_queries.py \
  --input collections/msmarco-passage/queries.train.tsv \
  --output collections/msmarco-ltr-passage/queries.train.json

#!gsutil cp -r collections/msmarco-ltr-passage/  gs://luanps/information_retrieval/pyserini/ltr/'''

In [13]:
!gsutil cp -r   gs://luanps/information_retrieval/pyserini/ltr/msmarco-ltr-passage/ collections/

Copying gs://luanps/information_retrieval/pyserini/ltr/msmarco-ltr-passage/ibm_model/body/output.t1.5.bin...
Copying gs://luanps/information_retrieval/pyserini/ltr/msmarco-ltr-passage/ibm_model/body/source.vcb...
Copying gs://luanps/information_retrieval/pyserini/ltr/msmarco-ltr-passage/ibm_model/body/target.vcb...
Copying gs://luanps/information_retrieval/pyserini/ltr/msmarco-ltr-passage/ibm_model/text_bert_tok/output.t1.5.bin...
\ [4 files][398.8 MiB/398.8 MiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://luanps/information_retrieval/pyserini/ltr/msmarco-ltr-passage/ibm_model/text_bert_tok/source.vcb...
Copying gs://luanps/information_retrieval/pyserini/ltr/msmarco-ltr-passage/ibm_model/text_bert_tok/target.vcb...
Copyin

### Run LTR inference

In [ ]:
!python -m pyserini.search.lucene.ltr  --input run.msmarco-passage.bm25tuned.txt \
  --input-format tsv \
  --model runs/msmarco-passage-ltr-mrr-v1 \
  --index msmarco-passage-ltr \
  --data passage \
  --ibm-model collections/msmarco-ltr-passage/ibm_model/ \
  --queries collections/msmarco-ltr-passage \
--output run.ltr.msmarco-passage.tsv 

Attempting to initialize pre-built index msmarco-passage-ltr.
index-msmarco-passage-ltr-20210519-e25e33f.tar.gz: 13.1GB [15:14, 15.4MB/s]                
Extracting /root/.cache/pyserini/indexes/index-msmarco-passage-ltr-20210519-e25e33f.tar.gz into /root/.cache/pyserini/indexes/index-msmarco-passage-ltr-20210519-e25e33f.a5de642c268ac1ed5892c069bdc29ae3...
Initializing msmarco-passage-ltr...
Attempting to initialize pre-built index msmarco-passage-ltr.
/root/.cache/pyserini/indexes/index-msmarco-passage-ltr-20210519-e25e33f.a5de642c268ac1ed5892c069bdc29ae3 already exists, skipping download.
Initializing msmarco-passage-ltr...
analyzed contents
text_unlemm text_unlemm
text_bert_tok text_bert_tok
IBM model Load takes 16.11 seconds
IBM model Load takes 30.96 seconds
IBM model Load takes 328.76 seconds
IBM model Load takes 99.07 seconds
---------------------loading dev----------------------------------------
(6974598, 2)
(6980,)
rank    999.226074
rel     999.226074
dtype: float64
        

In [ ]:
#MRR Eval
!python -m pyserini.eval.msmarco_passage_eval msmarco-passage-dev-subset run.ltr.msmarco-passage.tsv >> ltr_mrr_eval.txt



/usr/bin/python3: Error while finding module specification for 'pyserini.eval.msmarco_passage_eval' (ModuleNotFoundError: No module named 'pyserini')


In [ ]:
#TREC Eval
!python -m pyserini.eval.convert_msmarco_run_to_trec_run --input run.ltr.msmarco-passage.tsv --output run.ltr.msmarco-passage.trec

!python -m pyserini.eval.trec_eval -c -mrecall.1000 \
                                      -mmap msmarco-passage-dev-subset \
                                      run.ltr.msmarco-passage.trec >> ltr_trec_eval.txt

In [ ]:
!python -m pyserini.eval.convert_msmarco_run_to_trec_run \
   

!python -m pyserini/tools/scripts/msmarco/convert_msmarco_to_trec_qrels \
   --input tools/topics-and-qrels/qrels.msmarco-passage.dev-subset.txt --output collections/msmarco-passage/qrels.dev.small.trec

In [ ]:
$ tools/eval/trec_eval.9.0.4/trec_eval -c -mrecall.1000 -mmap \
   collections/msmarco-passage/qrels.dev.small.trec runs/run.ltr.msmarco-passage.trec
map                   	all	0.2551
recall_1000           	all	0.8573       	

### LTR Feature Extraction

In [15]:
!wc collections/msmarco-ltr-passage/queries.dev.small.json

   6980  231409 1786131 collections/msmarco-ltr-passage/queries.dev.small.json


In [18]:
!mkdir tmp
!head -n1 collections/msmarco-ltr-passage/queries.dev.small.json > tmp/queries.dev.small.json

In [17]:
!cat  queries.dev.small_1.json

{"id": "1048585", "text": "paula deen brother", "text_unlemm": "paula deen brother", "analyzed": "what paula deen brother", "entity": "{}", "raw": "what is paula deen's brother", "text_bert_tok": "what is paula dee ##n ' s brother"}


In [19]:
!python -m pyserini.search.lucene.ltr  --input run.msmarco-passage.bm25tuned.txt \
  --input-format tsv \
  --model runs/msmarco-passage-ltr-mrr-v1 \
  --index msmarco-passage-ltr \
  --data passage \
  --ibm-model collections/msmarco-ltr-passage/ibm_model/ \
  --queries collections/msmarco-ltr-passage \
--output run.ltr.msmarco-passage.tsv 

Attempting to initialize pre-built index msmarco-passage-ltr.
index-msmarco-passage-ltr-20210519-e25e33f.tar.gz: 13.1GB [15:35, 15.0MB/s]                
Extracting /root/.cache/pyserini/indexes/index-msmarco-passage-ltr-20210519-e25e33f.tar.gz into /root/.cache/pyserini/indexes/index-msmarco-passage-ltr-20210519-e25e33f.a5de642c268ac1ed5892c069bdc29ae3...
Initializing msmarco-passage-ltr...
Attempting to initialize pre-built index msmarco-passage-ltr.
/root/.cache/pyserini/indexes/index-msmarco-passage-ltr-20210519-e25e33f.a5de642c268ac1ed5892c069bdc29ae3 already exists, skipping download.
Initializing msmarco-passage-ltr...
analyzed contents
text_unlemm text_unlemm
text_bert_tok text_bert_tok
IBM model Load takes 15.46 seconds
IBM model Load takes 29.44 seconds
IBM model Load takes 309.44 seconds
IBM model Load takes 93.28 seconds
---------------------loading dev----------------------------------------
(6974598, 2)
(6980,)
rank    999.226074
rel     999.226074
dtype: float64
        

In [8]:
import sys

# We're going to explicitly use a local installation of Pyserini (as opposed to a pip-installed one).
# Comment these lines out to use a pip-installed one instead.
sys.path.insert(0, './')

import argparse
import json
import multiprocessing
import os
import pickle
import time

import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import defaultdict
#from pyserini.search.lucene.ltr._search_msmarco import MsmarcoLtrSearcher
#from pyserini.search.lucene.ltr import *

"""
Running prediction on candidates
"""
def dev_data_loader(file, format, data, top=100):
    if format == 'tsv':
        dev = pd.read_csv(file, sep="\t",
                          names=['qid', 'pid', 'rank'],
                          dtype={'qid': 'S','pid': 'S', 'rank':'i',})
    elif format == 'trec':
        dev = pd.read_csv(file, sep="\s+",
                    names=['qid', 'q0', 'pid', 'rank', 'score', 'tag'],
                    usecols=['qid', 'pid', 'rank'],
                    dtype={'qid': 'S','pid': 'S', 'rank':'i',})
    else:
        raise Exception('unknown parameters')
    assert dev['qid'].dtype == object
    assert dev['pid'].dtype == object
    assert dev['rank'].dtype == np.int32
    dev = dev[dev['rank']<=top]
    if data == 'passage':
        dev_qrel = pd.read_csv('tools/topics-and-qrels/qrels.msmarco-passage.dev-subset.txt', sep=" ",
                            names=["qid", "q0", "pid", "rel"], usecols=['qid', 'pid', 'rel'],
                            dtype={'qid': 'S','pid': 'S', 'rel':'i'})
    elif data == 'document':
        dev_qrel = pd.read_csv('tools/topics-and-qrels/qrels.msmarco-doc.dev.txt', sep="\t",
                            names=["qid", "q0", "pid", "rel"], usecols=['qid', 'pid', 'rel'],
                            dtype={'qid': 'S','pid': 'S', 'rel':'i'})
    assert dev['qid'].dtype == object
    assert dev['pid'].dtype == object
    assert dev['rank'].dtype == np.int32
    dev = dev.merge(dev_qrel, left_on=['qid', 'pid'], right_on=['qid', 'pid'], how='left')
    dev['rel'] = dev['rel'].fillna(0).astype(np.int32)
    dev = dev.sort_values(['qid', 'pid']).set_index(['qid', 'pid'])

    print(dev.shape)
    print(dev.index.get_level_values('qid').drop_duplicates().shape)
    print(dev.groupby('qid').count().mean())
    print(dev.head(10))
    print(dev.info())

    dev_rel_num = dev_qrel[dev_qrel['rel'] > 0].groupby('qid').count()['rel']

    recall_point = [10, 20, 50, 100, 200, 500, 1000, 2000, 5000, 10000]
    recall_curve = {k: [] for k in recall_point}
    for qid, group in tqdm(dev.groupby('qid')):
        group = group.reset_index()
        assert len(group['pid'].tolist()) == len(set(group['pid'].tolist()))
        total_rel = dev_rel_num.loc[qid]
        query_recall = [0 for k in recall_point]
        for t in group.sort_values('rank').itertuples():
            if t.rel > 0:
                for i, p in enumerate(recall_point):
                    if t.rank <= p:
                        query_recall[i] += 1
        for i, p in enumerate(recall_point):
            if total_rel > 0:
                recall_curve[p].append(query_recall[i] / total_rel)
            else:
                recall_curve[p].append(0.)

    for k, v in recall_curve.items():
        avg = np.mean(v)
        print(f'recall@{k}:{avg}')

    return dev, dev_qrel


def query_loader():
    queries = {}
    with open(f"{args['queries']}") as f:
        for line in f:
            query = json.loads(line)
            qid = query.pop('id')
            query['analyzed'] = query['analyzed'].split(" ")
            query['text'] = query['text_unlemm'].split(" ")
            query['text_unlemm'] = query['text_unlemm'].split(" ")
            query['text_bert_tok'] = query['text_bert_tok'].split(" ")
            queries[qid] = query
    return queries


def eval_mrr(dev_data):
    score_tie_counter = 0
    score_tie_query = set()
    MRR = []
    for qid, group in tqdm(dev_data.groupby('qid')):
        group = group.reset_index()
        rank = 0
        prev_score = None
        assert len(group['pid'].tolist()) == len(set(group['pid'].tolist()))
        # stable sort is also used in LightGBM

        for t in group.sort_values('score', ascending=False, kind='mergesort').itertuples():
            if prev_score is not None and abs(t.score - prev_score) < 1e-8:
                score_tie_counter += 1
                score_tie_query.add(qid)
            prev_score = t.score
            rank += 1
            if t.rel > 0:
                MRR.append(1.0 / rank)
                break
            elif rank == 10 or rank == len(group):
                MRR.append(0.)
                break

    score_tie = f'score_tie occurs {score_tie_counter} times in {len(score_tie_query)} queries'
    print(score_tie)
    mrr_10 = np.mean(MRR).item()
    print(f'MRR@10:{mrr_10} with {len(MRR)} queries')
    return {'score_tie': score_tie, 'mrr_10': mrr_10}


def eval_recall(dev_qrel, dev_data):
    dev_rel_num = dev_qrel[dev_qrel['rel'] > 0].groupby('qid').count()['rel']

    score_tie_counter = 0
    score_tie_query = set()

    recall_point = [10,20,50,100,200,250,300,333,400,500,1000]
    recall_curve = {k: [] for k in recall_point}
    for qid, group in tqdm(dev_data.groupby('qid')):
        group = group.reset_index()
        rank = 0
        prev_score = None
        assert len(group['pid'].tolist()) == len(set(group['pid'].tolist()))
        # stable sort is also used in LightGBM
        total_rel = dev_rel_num.loc[qid]
        query_recall = [0 for k in recall_point]
        for t in group.sort_values('score', ascending=False, kind='mergesort').itertuples():
            if prev_score is not None and abs(t.score - prev_score) < 1e-8:
                score_tie_counter += 1
                score_tie_query.add(qid)
            prev_score = t.score
            rank += 1
            if t.rel > 0:
                for i, p in enumerate(recall_point):
                    if rank <= p:
                        query_recall[i] += 1
        for i, p in enumerate(recall_point):
            if total_rel > 0:
                recall_curve[p].append(query_recall[i] / total_rel)
            else:
                recall_curve[p].append(0.)

    score_tie = f'score_tie occurs {score_tie_counter} times in {len(score_tie_query)} queries'
    print(score_tie)
    res = {'score_tie': score_tie}

    for k, v in recall_curve.items():
        avg = np.mean(v)
        print(f'recall@{k}:{avg}')
        res[f'recall@{k}'] = avg

    return res

In [57]:
import logging
from typing import Dict, List, Optional, Union
import multiprocessing
import time
import os
from tqdm import tqdm
import pickle
from pyserini.index import IndexReader
from pyserini.search.lucene import LuceneSearcher
from pyserini.util import get_cache_home

from pyserini.search.lucene.ltr._base import *


logger = logging.getLogger(__name__)

class MsmarcoLtrSearcherFeatExtractor:
    def __init__(self, model: str, ibm_model:str, index:str, data: str):
        #msmarco-ltr-passage
        self.model = model
        self.ibm_model = ibm_model
        self.lucene_searcher = LuceneSearcher.from_prebuilt_index(index)
        index_directory = os.path.join(get_cache_home(), 'indexes')
        if (data == 'passage'):
            index_path = os.path.join(index_directory, 'index-msmarco-passage-ltr-20210519-e25e33f.a5de642c268ac1ed5892c069bdc29ae3')
        else:
            index_path = os.path.join(index_directory, 'index-msmarco-doc-per-passage-ltr-20211031-33e4151.bd60e89041b4ebbabc4bf0cfac608a87')
        self.fe = FeatureExtractor(index_path, max(multiprocessing.cpu_count()//2, 1))
        self.index_reader = IndexReader.from_prebuilt_index(index)
        self.data = data

    
    def add_fe(self):
        #self.fe.add(RunList('collections/msmarco-ltr-passage/run.monot5.run_list.whole.trec','t5'))
        #self.fe.add(RunList('../bert.whole.doc.trec','bert'))
        for qfield, ifield in [('analyzed', 'contents'),
                           ('text_unlemm', 'text_unlemm'),
                           ('text_bert_tok', 'text_bert_tok')]:
            print(qfield, ifield)
            self.fe.add(BM25Stat(SumPooler(), k1=2.0, b=0.75, field=ifield, qfield=qfield))
            self.fe.add(BM25Stat(AvgPooler(), k1=2.0, b=0.75, field=ifield, qfield=qfield))
            self.fe.add(BM25Stat(MedianPooler(), k1=2.0, b=0.75, field=ifield, qfield=qfield))
            self.fe.add(BM25Stat(MaxPooler(), k1=2.0, b=0.75, field=ifield, qfield=qfield))
            self.fe.add(BM25Stat(MinPooler(), k1=2.0, b=0.75, field=ifield, qfield=qfield))
            self.fe.add(BM25Stat(MaxMinRatioPooler(), k1=2.0, b=0.75, field=ifield, qfield=qfield))

            self.fe.add(LmDirStat(SumPooler(), mu=1000, field=ifield, qfield=qfield))
            self.fe.add(LmDirStat(AvgPooler(), mu=1000, field=ifield, qfield=qfield))
            self.fe.add(LmDirStat(MedianPooler(), mu=1000, field=ifield, qfield=qfield))
            self.fe.add(LmDirStat(MaxPooler(), mu=1000, field=ifield, qfield=qfield))
            self.fe.add(LmDirStat(MinPooler(), mu=1000, field=ifield, qfield=qfield))
            self.fe.add(LmDirStat(MaxMinRatioPooler(), mu=1000, field=ifield, qfield=qfield))

            self.fe.add(NormalizedTfIdf(field=ifield, qfield=qfield))
            self.fe.add(ProbalitySum(field=ifield, qfield=qfield))

            self.fe.add(DfrGl2Stat(SumPooler(), field=ifield, qfield=qfield))
            self.fe.add(DfrGl2Stat(AvgPooler(), field=ifield, qfield=qfield))
            self.fe.add(DfrGl2Stat(MedianPooler(), field=ifield, qfield=qfield))
            self.fe.add(DfrGl2Stat(MaxPooler(), field=ifield, qfield=qfield))
            self.fe.add(DfrGl2Stat(MinPooler(), field=ifield, qfield=qfield))
            self.fe.add(DfrGl2Stat(MaxMinRatioPooler(), field=ifield, qfield=qfield))

            self.fe.add(DfrInExpB2Stat(SumPooler(), field=ifield, qfield=qfield))
            self.fe.add(DfrInExpB2Stat(AvgPooler(), field=ifield, qfield=qfield))
            self.fe.add(DfrInExpB2Stat(MedianPooler(), field=ifield, qfield=qfield))
            self.fe.add(DfrInExpB2Stat(MaxPooler(), field=ifield, qfield=qfield))
            self.fe.add(DfrInExpB2Stat(MinPooler(), field=ifield, qfield=qfield))
            self.fe.add(DfrInExpB2Stat(MaxMinRatioPooler(), field=ifield, qfield=qfield))

            self.fe.add(DphStat(SumPooler(), field=ifield, qfield=qfield))
            self.fe.add(DphStat(AvgPooler(), field=ifield, qfield=qfield))
            self.fe.add(DphStat(MedianPooler(), field=ifield, qfield=qfield))
            self.fe.add(DphStat(MaxPooler(), field=ifield, qfield=qfield))
            self.fe.add(DphStat(MinPooler(), field=ifield, qfield=qfield))
            self.fe.add(DphStat(MaxMinRatioPooler(), field=ifield, qfield=qfield))

            self.fe.add(Proximity(field=ifield, qfield=qfield))
            self.fe.add(TpScore(field=ifield, qfield=qfield))
            self.fe.add(TpDist(field=ifield, qfield=qfield))

            self.fe.add(DocSize(field=ifield))

            self.fe.add(QueryLength(qfield=qfield))
            self.fe.add(QueryCoverageRatio(qfield=qfield))
            self.fe.add(UniqueTermCount(qfield=qfield))
            self.fe.add(MatchingTermCount(field=ifield, qfield=qfield))
            self.fe.add(SCS(field=ifield, qfield=qfield))

            self.fe.add(TfStat(AvgPooler(), field=ifield, qfield=qfield))
            self.fe.add(TfStat(MedianPooler(), field=ifield, qfield=qfield))
            self.fe.add(TfStat(SumPooler(), field=ifield, qfield=qfield))
            self.fe.add(TfStat(MinPooler(), field=ifield, qfield=qfield))
            self.fe.add(TfStat(MaxPooler(), field=ifield, qfield=qfield))
            self.fe.add(TfStat(MaxMinRatioPooler(), field=ifield, qfield=qfield))

            self.fe.add(TfIdfStat(True, AvgPooler(), field=ifield, qfield=qfield))
            self.fe.add(TfIdfStat(True, MedianPooler(), field=ifield, qfield=qfield))
            self.fe.add(TfIdfStat(True, SumPooler(), field=ifield, qfield=qfield))
            self.fe.add(TfIdfStat(True, MinPooler(), field=ifield, qfield=qfield))
            self.fe.add(TfIdfStat(True, MaxPooler(), field=ifield, qfield=qfield))
            self.fe.add(TfIdfStat(True, MaxMinRatioPooler(), field=ifield, qfield=qfield))

            self.fe.add(NormalizedTfStat(AvgPooler(), field=ifield, qfield=qfield))
            self.fe.add(NormalizedTfStat(MedianPooler(), field=ifield, qfield=qfield))
            self.fe.add(NormalizedTfStat(SumPooler(), field=ifield, qfield=qfield))
            self.fe.add(NormalizedTfStat(MinPooler(), field=ifield, qfield=qfield))
            self.fe.add(NormalizedTfStat(MaxPooler(), field=ifield, qfield=qfield))
            self.fe.add(NormalizedTfStat(MaxMinRatioPooler(), field=ifield, qfield=qfield))

            self.fe.add(IdfStat(AvgPooler(), field=ifield, qfield=qfield))
            self.fe.add(IdfStat(MedianPooler(), field=ifield, qfield=qfield))
            self.fe.add(IdfStat(SumPooler(), field=ifield, qfield=qfield))
            self.fe.add(IdfStat(MinPooler(), field=ifield, qfield=qfield))
            self.fe.add(IdfStat(MaxPooler(), field=ifield, qfield=qfield))
            self.fe.add(IdfStat(MaxMinRatioPooler(), field=ifield, qfield=qfield))

            self.fe.add(IcTfStat(AvgPooler(), field=ifield, qfield=qfield))
            self.fe.add(IcTfStat(MedianPooler(), field=ifield, qfield=qfield))
            self.fe.add(IcTfStat(SumPooler(), field=ifield, qfield=qfield))
            self.fe.add(IcTfStat(MinPooler(), field=ifield, qfield=qfield))
            self.fe.add(IcTfStat(MaxPooler(), field=ifield, qfield=qfield))
            self.fe.add(IcTfStat(MaxMinRatioPooler(), field=ifield, qfield=qfield))

            self.fe.add(UnorderedSequentialPairs(3, field=ifield, qfield=qfield))
            self.fe.add(UnorderedSequentialPairs(8, field=ifield, qfield=qfield))
            self.fe.add(UnorderedSequentialPairs(15, field=ifield, qfield=qfield))
            self.fe.add(OrderedSequentialPairs(3, field=ifield, qfield=qfield))
            self.fe.add(OrderedSequentialPairs(8, field=ifield, qfield=qfield))
            self.fe.add(OrderedSequentialPairs(15, field=ifield, qfield=qfield))
            self.fe.add(UnorderedQueryPairs(3, field=ifield, qfield=qfield))
            self.fe.add(UnorderedQueryPairs(8, field=ifield, qfield=qfield))
            self.fe.add(UnorderedQueryPairs(15, field=ifield, qfield=qfield))
            self.fe.add(OrderedQueryPairs(3, field=ifield, qfield=qfield))
            self.fe.add(OrderedQueryPairs(8, field=ifield, qfield=qfield))
            self.fe.add(OrderedQueryPairs(15, field=ifield, qfield=qfield))

        start = time.time()
        self.fe.add(IbmModel1(f"{self.ibm_model}/title_unlemm", "text_unlemm", "title_unlemm", "text_unlemm"))
        end = time.time()
        print('IBM model Load takes %.2f seconds' % (end - start))
        start = end
        self.fe.add(IbmModel1(f"{self.ibm_model}url_unlemm", "text_unlemm", "url_unlemm", "text_unlemm"))
        end = time.time()
        print('IBM model Load takes %.2f seconds' % (end - start))
        start = end
        self.fe.add(IbmModel1(f"{self.ibm_model}body", "text_unlemm", "body", "text_unlemm"))
        end = time.time()
        print('IBM model Load takes %.2f seconds' % (end - start))
        start = end
        self.fe.add(IbmModel1(f"{self.ibm_model}text_bert_tok", "text_bert_tok", "text_bert_tok", "text_bert_tok"))
        end = time.time()
        print('IBM model Load takes %.2f seconds' % (end - start))
        start = end
    
    def batch_extract(self, df, queries, fe):
        tasks = []
        task_infos = []
        group_lst = []

        for qid, group in tqdm(df.groupby('qid')):
            task = {
                "qid": qid,
                "docIds": [],
                "rels": [],
                "query_dict": queries[qid]
            }
            for t in group.reset_index().itertuples():
                if (self.data == 'document'):
                    if (self.index_reader.doc(t.pid) != None):
                        task["docIds"].append(t.pid)
                        task_infos.append((qid, t.pid, t.rel))
                else:
                    task["docIds"].append(t.pid)
                    task_infos.append((qid, t.pid, t.rel))
            tasks.append(task)
            group_lst.append((qid, len(task['docIds'])))
            if len(tasks) == 1000:
                features = fe.batch_extract(tasks)
                task_infos = pd.DataFrame(task_infos, columns=['qid', 'pid', 'rel'])
                group = pd.DataFrame(group_lst, columns=['qid', 'count'])
                print(features.shape)
                print(task_infos.qid.drop_duplicates().shape)
                print(group.mean())
                print(features.head(10))
                print(features.info())
                yield task_infos, features, group
                tasks = []
                task_infos = []
                group_lst = []
        # deal with rest
        if len(tasks) > 0:
            features = fe.batch_extract(tasks)
            task_infos = pd.DataFrame(task_infos, columns=['qid', 'pid', 'rel'])
            group = pd.DataFrame(group_lst, columns=['qid', 'count'])
            print(features.shape)
            print(task_infos.qid.drop_duplicates().shape)
            print(group.mean())
            print(features.head(10))
            print(features.info())
            yield task_infos, features, group

        return

    def batch_predict(self, models, dev_extracted, feature_name):
        task_infos, features, group = dev_extracted
        dev_X = features.loc[:, feature_name]

        task_infos['score'] = 0.
        for gbm in models:
            task_infos['score'] += gbm.predict(dev_X)
    
    def search(self, dev, queries):
        batch_info = []
        start_extract = time.time()
        models = pickle.load(open(self.model+'/model.pkl', 'rb'))
        metadata = json.load(open(self.model+'/metadata.json', 'r'))
        feature_used = metadata['feature_names']
        for dev_extracted in self.batch_extract(dev, queries, self.fe):
            end_extract = time.time()
            print(f'extract 1000 queries take {end_extract - start_extract}s')
            task_infos, features, group = dev_extracted
            start_predict = time.time()
            self.batch_predict(models, dev_extracted, feature_used)
            end_predict = time.time()
            print(f'predict 1000 queries take {end_predict - start_predict}s')
            batch_info.append(task_infos)
            start_extract = time.time()
        batch_info = pd.concat(batch_info, axis=0, ignore_index=True)
        return batch_info



In [3]:
args = {
    'input': 'run.msmarco-passage.bm25tuned.txt',
    'input_format': 'tsv',
    'model': 'runs/msmarco-passage-ltr-mrr-v1',
    'index': 'msmarco-passage-ltr',
    'data': 'passage',
    'ibm_model': 'collections/msmarco-ltr-passage/ibm_model/',
    'queries':  'collections/msmarco-ltr-passage/queries.dev.small.json',
    'output': 'run.ltr.msmarco-passage.tsv',
    'reranking_top': 10000,
    'max_passage': True
  }

In [9]:
dev, dev_qrel = dev_data_loader(args['input'], args['input_format'], args['data'], args['reranking_top'])
queries = query_loader()

(6974598, 2)
(6980,)
rank    999.226074
rel     999.226074
dtype: float64
                 rank  rel
qid     pid               
1000000 1037824   173    0
        1037828   813    0
        1037830   912    0
        1037832   537    0
        1072976   197    0
        107330    626    0
        1077279   200    0
        1115562   760    0
        1121786   866    0
        1124405   994    0
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 6974598 entries, ('1000000', '1037824') to ('999942', '996294')
Data columns (total 2 columns):
 #   Column  Dtype
---  ------  -----
 0   rank    int32
 1   rel     int32
dtypes: int32(2)
memory usage: 251.6+ MB
None


100%|██████████| 6980/6980 [01:50<00:00, 62.97it/s]


recall@10:0.39159503342884433
recall@20:0.48416905444126074
recall@50:0.5923352435530086
recall@100:0.6701408787010505
recall@200:0.7382760267430755
recall@500:0.8115568290353391
recall@1000:0.8573424068767909
recall@2000:0.8573424068767909
recall@5000:0.8573424068767909
recall@10000:0.8573424068767909


In [1]:
from pyserini.search.lucene.ltr import MsmarcoLtrSearcher

In [4]:
searcher = MsmarcoLtrSearcher(args['model'], args['ibm_model'], args['index'], args['data'])

Attempting to initialize pre-built index msmarco-passage-ltr.
/root/.cache/pyserini/indexes/index-msmarco-passage-ltr-20210519-e25e33f.a5de642c268ac1ed5892c069bdc29ae3 already exists, skipping download.
Initializing msmarco-passage-ltr...
Attempting to initialize pre-built index msmarco-passage-ltr.
/root/.cache/pyserini/indexes/index-msmarco-passage-ltr-20210519-e25e33f.a5de642c268ac1ed5892c069bdc29ae3 already exists, skipping download.
Initializing msmarco-passage-ltr...


In [5]:
searcher.add_fe()

#batch_info = searcher.search(dev, queries)
#del dev, queries

#eval_res = eval_mrr(batch_info)
#eval_recall(dev_qrel, batch_info)
#output(args.output, batch_info,args.output_format, args.max_passage)


analyzed contents
text_unlemm text_unlemm
text_bert_tok text_bert_tok
IBM model Load takes 14.63 seconds
IBM model Load takes 35.10 seconds
IBM model Load takes 311.57 seconds
IBM model Load takes 94.46 seconds


In [23]:
dev.count()

rank    6974598
rel     6974598
dtype: int64

In [10]:
extracted = searcher.batch_extract(dev, queries, searcher.fe)

In [11]:
for i in extracted:
  print(i)
  break

100%|██████████| 1/1 [00:00<00:00, 65.95it/s]


(1000, 253)
(1,)
qid      1000000.0
count       1000.0
dtype: float64
   contents_analyzed_BM25Stat_sum_k1_2.00_b_0.75  ...  text_bert_tok_text_bert_tok_IbmModel1_text_bert_tok
0                                      18.272890  ...                                          22.790607  
1                                      16.755219  ...                                          21.892771  
2                                      15.543254  ...                                          22.036352  
3                                      16.454018  ...                                          21.337503  
4                                      17.806166  ...                                          22.189713  
5                                      16.702007  ...                                          21.950027  
6                                      17.989443  ...                                          22.597744  
7                                      13.815506  ...                     

In [16]:
task_infos, features, group = i

In [18]:
features

,contents_analyzed_BM25Stat_sum_k1_2.00_b_0.75,contents_analyzed_BM25Stat_avg_k1_2.00_b_0.75,contents_analyzed_BM25Stat_median_k1_2.00_b_0.75,contents_analyzed_BM25Stat_max_k1_2.00_b_0.75,contents_analyzed_BM25Stat_min_k1_2.00_b_0.75,contents_analyzed_BM25Stat_maxminratio_k1_2.00_b_0.75,contents_analyzed_LmDirStat_sum_mu_1000,contents_analyzed_LmDirStat_avg_mu_1000,contents_analyzed_LmDirStat_median_mu_1000,contents_analyzed_LmDirStat_max_mu_1000,contents_analyzed_LmDirStat_min_mu_1000,contents_analyzed_LmDirStat_maxminratio_mu_1000,contents_analyzed_NormalizedTfIdfF,contents_analyzed_ProbalitySum,contents_analyzed_DfrGl2Stat_sum,contents_analyzed_DfrGl2Stat_avg,contents_analyzed_DfrGl2Stat_median,contents_analyzed_DfrGl2Stat_max,contents_analyzed_DfrGl2Stat_min,contents_analyzed_DfrGl2Stat_maxminratio,contents_analyzed_DfrInExpB2Stat_sum,contents_analyzed_DfrInExpB2Stat_avg,contents_analyzed_DfrInExpB2Stat_median,contents_analyzed_DfrInExpB2Stat_max,contents_analyzed_DfrInExpB2Stat_min,contents_analyzed_DfrInExpB2Stat_maxminratio,contents_analyzed_DphStat_sum,contents_analyzed_DphStat_avg,contents_analyzed_DphStat_median,contents_analyzed_DphStat_max,contents_analyzed_DphStat_min,contents_analyzed_DphStat_maxminratio,contents_analyzed_Proximity,contents_analyzed_TpScore,contents_analyzed_TpDist,contents_DocSize,analyzed_QueryLength,contents_analyzed_QueryCoverageRatio,analyzed_UniqueTermCount,contents_analyzed_MatchingTermCount,...,text_bert_tok_text_bert_tok_LTfIdfStat_avg,text_bert_tok_text_bert_tok_LTfIdfStat_median,text_bert_tok_text_bert_tok_LTfIdfStat_sum,text_bert_tok_text_bert_tok_LTfIdfStat_min,text_bert_tok_text_bert_tok_LTfIdfStat_max,text_bert_tok_text_bert_tok_LTfIdfStat_maxminratio,text_bert_tok_text_bert_tok_NormalizedTfStat_avg,text_bert_tok_text_bert_tok_NormalizedTfStat_median,text_bert_tok_text_bert_tok_NormalizedTfStat_sum,text_bert_tok_text_bert_tok_NormalizedTfStat_min,text_bert_tok_text_bert_tok_NormalizedTfStat_max,text_bert_tok_text_bert_tok_NormalizedTfStat_maxminratio,text_bert_tok_text_bert_tok_IdfStat_avg,text_bert_tok_text_bert_tok_IdfStat_median,text_bert_tok_text_bert_tok_IdfStat_sum,text_bert_tok_text_bert_tok_IdfStat_min,text_bert_tok_text_bert_tok_IdfStat_max,text_bert_tok_text_bert_tok_IdfStat_maxminratio,text_bert_tok_text_bert_tok_IcTfStat_avg,text_bert_tok_text_bert_tok_IcTfStat_median,text_bert_tok_text_bert_tok_IcTfStat_sum,text_bert_tok_text_bert_tok_IcTfStat_min,text_bert_tok_text_bert_tok_IcTfStat_max,text_bert_tok_text_bert_tok_IcTfStat_maxminratio,text_bert_tok_text_bert_tok_UnorderedSequentialPairs_with_gap_3,text_bert_tok_text_bert_tok_UnorderedSequentialPairs_with_gap_8,text_bert_tok_text_bert_tok_UnorderedSequentialPairs_with_gap_15,text_bert_tok_text_bert_tok_OrderedSequentialPairs_3,text_bert_tok_text_bert_tok_OrderedSequentialPairs_8,text_bert_tok_text_bert_tok_OrderedSequentialPairs_15,text_bert_tok_text_bert_tok_UnorderedQueryPairs_with_gap_3,text_bert_tok_text_bert_tok_UnorderedQueryPairs_with_gap_8,text_bert_tok_text_bert_tok_UnorderedQueryPairs_with_gap_15,text_bert_tok_text_bert_tok_OrderedQueryPairs_3,text_bert_tok_text_bert_tok_OrderedQueryPairs_8,text_bert_tok_text_bert_tok_OrderedQueryPairs_15,text_unlemm_text_unlemm_IbmModel1_title_unlemm,text_unlemm_text_unlemm_IbmModel1_url_unlemm,text_unlemm_text_unlemm_IbmModel1_body,text_bert_tok_text_bert_tok_IbmModel1_text_bert_tok
0,18.272890,3.045482,1.254197,12.133188,0.0,4.837035,-38.989208,-6.498201,-6.668258,1.401298e-45,-7.934131,-0.210341,0.576942,0.176471,7.312206,2.437402,0.870229,4.437000,1.134748,3.910121,17.497530,5.832510,1.424856,12.749043,1.898776,6.714349,8.314975,2.771658,1.009035,4.667369,1.629535,2.864233,-8.457381,16.441662,0.0,34.0,6.0,0.500000,6.0,3.0,...,3.197732,1.173600,19.186390,0.0,13.506985,5.754508,4.125485,4.424111,24.752909,2.978925,4.770685,1.409614,3.93053,3.909062,23.583181,1.579744,6.436404,4.074334,7.97247,8.071041,47.83482,5.606049,10.053039,1.793249,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0

In [ ]:
batch_info = []

for dev_extracted in searcher.batch_extract(dev, queries, searcher.fe):
    #task_infos, features, group = dev_extracted
    #searcher.batch_predict(models, dev_extracted, feature_used)
    batch_info.append(dev_extracted)
#batch_info = pd.concat(batch_info, axis=0, ignore_index=True)
pickle.dump(batch_info,open('msmarco_passage_dev.bm25tuned.features.pkl','wb'))

 14%|█▍        | 997/6980 [00:28<01:34, 63.08it/s]

In [ ]:
!mv msmarco_passage_dev.bm25tuned.features msmarco_passage_dev.bm25tuned.features.pkl
!gsutil cp msmarco_passage_dev.bm25tuned.features.pkl gs://luanps/information_retrieval/pyserini/ltr

In [ ]:
!ls